In [3]:
#数据准备

In [5]:
import numpy as np 
import pandas as pd 
train = pd.read_csv('/Users/senlinlidewo/金融风控比赛/用户逾期/train.csv')
test = pd.read_csv('/Users/senlinlidewo/金融风控比赛/用户逾期/test.csv')


In [8]:
# 文字类列处理
print(train.IDF_TYP_CD.unique())
train.IDF_TYP_CD = train.IDF_TYP_CD.apply(lambda x:x.strip())
train['IDF_TYP_CD'] = [i.replace('ZR',' ') for i in train.IDF_TYP_CD]
#去除某列特定字符。

['01' '20' '03' '09' '22' '21' '04' '07' '11' '16' '06' '05' '08' '02'
 '13' '14' '10' '12']


In [11]:
print(test.IDF_TYP_CD.unique())
test.IDF_TYP_CD = test.IDF_TYP_CD.apply(lambda x:x.strip())
test['IDF_TYP_CD'] = [i.replace('ZR',' ') for i in test.IDF_TYP_CD]
#去除某列特定字符。

['01' '20' '03' '09' '21' '07' '11' '12' '22' '04' '05' '08' '06' '13'
 '10' '02']


In [12]:
col = list(train.columns)
train[col] = train[col].apply(pd.to_numeric,errors='coerce').fillna(0.0)
train = pd.DataFrame(train,dtype='float')

In [13]:
train1 = train.drop(columns=["bad_good"])

In [10]:
########## LogisticRegression ############

In [28]:
X_train = train.iloc[:,1:50]
y_train = train["bad_good"]
X_test = test
X_train.shape

(285285, 49)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score

In [34]:
#https://blog.csdn.net/jagbiam1000/article/details/79764012?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161459589616780262584323%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161459589616780262584323&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-79764012.pc_search_result_hbase_insert&utm_term=logisticregression+%E5%8F%82%E6%95%B0
#https://blog.csdn.net/ztf312/article/details/98521376

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train1,train["bad_good"],test_size = 0.33,random_state=0)
len(X_train)

191140

In [16]:
logreg = LogisticRegression(solver='sag',
            penalty= 'l2', random_state = 50
                                
                            )

In [17]:
logreg.fit(X_train,y_train)

/Users/senlinlidewo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(random_state=50, solver='sag')

In [18]:
y_pred = logreg.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.982686281799352

In [21]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)
#Precision: How often is it correct when the model predicts yes.

0.0

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
# printing confision matrix
confusion = pd.DataFrame(confusion_matrix(y_test,y_pred),\
            columns=["Predicted good", "Predicted bad"],\
            index=["good","bad"] )
confusion

,Predicted good,Predicted bad
good,92515,8
bad,1622,0


In [56]:
#这个模型没有收敛，只要把所有的坏人都认为好的就可以有很高的正确率，因为好人占大部分。

In [ ]:
#True Positive(TP):87
#True Negative(TN):149
#False Positive(FP):28
#False Negative(FN):30
#From these four terminologies, we can compute many other rates that are used to evaluate a binary classifier.
#Accuracy:
#Accuracy is the measure of how often the model is correct.

#(TP + TN)/total = (87+149)/294 = .8027
#好人通过率：好/真好；坏人放过率：坏人/真坏人

In [22]:
#########GridSearchCV#########

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
C_vals = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
cv = StratifiedShuffleSplit(n_splits = 10, test_size = .25)
param = {'C': C_vals}
logreg = LogisticRegression()
## Calling on GridSearchCV object. 
grid = GridSearchCV(
    estimator=LogisticRegression(), 
    param_grid = param,
    scoring = 'accuracy',
    n_jobs =-1,
    cv = cv
)
## Fitting the model
grid.fit(X_train,y_train)

KeyboardInterrupt: 

In [30]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'